<a href="https://colab.research.google.com/github/aadrijupadya/NBA-Playoff-Prediction-Model/blob/main/Playoff_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from xgboost import XGBRegressor

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

Importing the necessary libraries to complete this project. Used sklearn to analyze the model, seaborn and numpy for graphing, and xgboost for the actual algorithm

In [3]:
df_2021 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2020-2021.csv")
df_2021
df_2020 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2019-2020.csv")
df_2020
df_2019 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2018-2019.csv")
df_2019
df_2018 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2017-2018.csv")
df_2018
df_2017 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2016-2017.csv")
df_2017
df_2016 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2015-2016.csv")
df_2016
df_2015 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/2014-2015.csv")
df_2015
#loading the data

df_2021['Year'] = [2021 for x in range(len(df_2021.index))]
df_2020['Year'] = [2020 for x in range(len(df_2020.index))]
df_2019['Year'] = [2019 for x in range(len(df_2019.index))]
df_2018['Year'] = [2018 for x in range(len(df_2018.index))]
df_2017['Year'] = [2017 for x in range(len(df_2017.index))]
df_2016['Year'] = [2016 for x in range(len(df_2016.index))]
df_2015['Year'] = [2015 for x in range(len(df_2015.index))]
#Adding Year

frames = [df_2021, df_2020, df_2019, df_2018, df_2017, df_2016, df_2015]
metadata = pd.concat(frames)
#concatenating into one large dataframe

metadata = metadata.drop(columns=['Unnamed: 17', 'Unnamed: 22', 'Unnamed: 27'])
metadata['Index'] = [x for x in range(len(metadata.index))]
metadata = metadata.set_index('Index')
#Adding an Index that can be used for accessing specific rows

cnt = 0
for team in metadata['Team']:
    if '*' not in str(team):
        metadata = metadata.drop(cnt)
    cnt+=1
#cutting so that only playoff teams remain

metadata['Index'] = [x for x in range(len(metadata.index))]
metadata = metadata.set_index('Index')


from IPython.display import display
with pd.option_context('display.max_rows', 250):
    display(metadata)

metadata['Playoff Wins'] = [6,10,14,16,7,4,7,2,3,1,10,2,1,1,0,1,5,10,7,7,16,2,14,5,3,0,9,0,3,0,1,1,10,14,16,1,6,5,8,7,0,1,7,3,2,1,1,0,11,4,16,5,5,11,2,1,0,1,3,5,12,2,1,3,16,8,6,4,3,4,13,9,7,1,2,2,0,2,0,2,6,15,11,16,10,2,4,2,3,7,3,5,0,1,1,0,16,7,3,8,14,1,9,6,0,6,1,0,6,2,0,2]
metadata['Playoff Win Percentage'] = [0.545,0.526,0.636,0.696,0.583,0.400,0.583,0.333,0.429,0.200,0.556,0.333,0.200,0.200,0,0.200,0.500,0.588,0.538,0.636,0.762,0.333,0.667,0.417,0.429,0,0.474,0,0.429,0,0.200,0.200,0.667,0.636,0.667,0.200,0.545,0.556,0.500,0.500,0,0.200,0.583,0.429,0.333,0.200,0.200,0,0.647,0.400,0.762,0.455,0.500,0.579,0.333,0.200,0,0.200,0.429,0.556,0.545,0.333,0.200,0.429,0.941,0.500,0.545,0.400,0.429,0.364,0.722,0.500,0.538,0.200,0.333,0.333,0,0.333,0,0.333,0.600,0.625,0.611,0.762,0.500,0.333,0.400,0.333,0.429,0.500,0.429,0.455,0,0.200,0.200,0,0.762,0.500,0.429,0.500,0.700,0.200,0.529,0.545,0,0.500,0.200,0,0.600,0.333,0,0.33]
metadata
#Adding Wins and Playoff Win Percentage


,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,ORB%,FT/FGA,deFG%,dTOV%,dDRB%,dFT/FGA,Arena,Attend.,Attend./G,Year
Index,,,,,,,,,,,,,,,,,,,,,
0,1.0,Utah Jazz*,28.5,52.0,20.0,55,17,9.25,-0.29,8.97,...,24.5,0.195,0.507,10.3,79.3,0.159,Vivint Smart Home Arena,151300.0,4203.0,2021
1,2.0,Los Angeles Clippers*,28.8,47.0,25.0,49,23,6.18,-0.16,6.02,...,22.7,0.186,0.531,11.9,79.1,0.186,STAPLES Center,13901.0,386.0,2021
2,3.0,Phoenix Suns*,26.6,51.0,21.0,49,23,5.82,-0.15,5.67,...,20.8,0.177,0.534,12.4,78.5,0.194,Phoenix Suns Arena,104027.0,2890.0,2021
3,4.0,Milwaukee Bucks*,28.1,46.0,26.0,48,24,5.89,-0.32,5.57,...,23.3,0.177,0.536,11.5,79.7,0.157,Fiserv Forum,64780.0,1799.0,2021
4,5.0,Philadelphia 76ers*,27.1,49.0,23.0,48,24,5.58,-0.31,5.28,...,23.2,0.225,0.521,13.8,78.2,0.200,Wells Fargo Center,68583.0,1905.0,2021
5,6.0,Denver Nuggets*,26.1,47.0,25.0,47,25,4.93,-0.11,4.82,...,24.7,0.176,0.545,12.9,78.9,0.200,Ball Arena,54563.0,1516.0,2021
6,7.0,Brooklyn Nets*,28.2,48.0,24.0,46,26,4.50,-0.27,4.24,...,21.4,0.208,0.531,11.1,77.3,0.187,Barclays Center,30491.0,847.0,2021
7,8.0,Los Angeles Lakers*,28.2,42.0,30.0,42,30,2.79,-0.03,2.77,...,22.5,0.200,0.526,13.7,79.7,0.184,STAPLES Center,23313.0,648.0,2021
8,9.0,Dallas Mavericks*,26.3,42.0,30.0,41,31,2.26,-0.01,2.26,...,21.1,0.189,0.534,11.5,77.8,0.197,American Airlines Center,94849.0,2635.0,2021


,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,deFG%,dTOV%,dDRB%,dFT/FGA,Arena,Attend.,Attend./G,Year,Playoff Wins,Playoff Win Percentage
Index,,,,,,,,,,,,,,,,,,,,,
0,1.0,Utah Jazz*,28.5,52.0,20.0,55,17,9.25,-0.29,8.97,...,0.507,10.3,79.3,0.159,Vivint Smart Home Arena,151300.0,4203.0,2021,6,0.545
1,2.0,Los Angeles Clippers*,28.8,47.0,25.0,49,23,6.18,-0.16,6.02,...,0.531,11.9,79.1,0.186,STAPLES Center,13901.0,386.0,2021,10,0.526
2,3.0,Phoenix Suns*,26.6,51.0,21.0,49,23,5.82,-0.15,5.67,...,0.534,12.4,78.5,0.194,Phoenix Suns Arena,104027.0,2890.0,2021,14,0.636
3,4.0,Milwaukee Bucks*,28.1,46.0,26.0,48,24,5.89,-0.32,5.57,...,0.536,11.5,79.7,0.157,Fiserv Forum,64780.0,1799.0,2021,16,0.696
4,5.0,Philadelphia 76ers*,27.1,49.0,23.0,48,24,5.58,-0.31,5.28,...,0.521,13.8,78.2,0.200,Wells Fargo Center,68583.0,1905.0,2021,7,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,13.0,New Orleans Pelicans*,25.3,45.0,37.0,43,39,0.79,0.34,1.13,...,0.496,11.7,75.1,0.195,Smoothie King Center,683757.0,16677.0,2015,0,0.000
108,14.0,Washington Wizards*,28.6,46.0,36.0,43,39,0.72,-0.55,0.17,...,0.481,12.9,77.3,0.210,Verizon Center,747796.0,18239.0,2015,6,0.600
109,15.0,Milwaukee Bucks*,24.6,41.0,41.0,42,40,0.43,-0.52,-0.09,...,0.487,15.9,73.3,0.221,BMO Harris Bradley Center,593910.0,14813.0,2015,2,0.333


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
non_str_data = metadata.drop(columns=['Team','Arena','Year'])
metadata.to_csv('metadata.csv')
np.set_printoptions(suppress=True)
non_str_data.to_csv('non_str_data_2.csv')


In [5]:
dataset = loadtxt('/content/drive/MyDrive/Colab Notebooks/non_str_data_2.csv',delimiter=',',skiprows=1,usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,27,28])
X = dataset[1:,:24]
Y = dataset[1:,25:26]
#loading data into numpy arrays


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#training by splitting into test arrays

In [7]:
model = XGBRegressor()
model.fit(X_train, y_train)
print(model)
#XGB Regression model

[06:01:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor()


In [8]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)
predictions = [round(value,2) for value in y_pred]
print(predictions) #set of predicted win percentage values
print(mean_squared_error(y_test,predictions)) #measure of accuracy
print(X_test) #test input parameters

[0.37, 0.22, 0.57, 0.59, 0.56, 0.45, 0.3, 0.19, 0.52, 0.2, 0.74, 0.77, 0.63, 0.54, 0.72, 0.62, 0.43, 0.39, 0.1, 0.42, 0.25, 0.37, 0.61, 0.53, 0.54, 0.42, 0.81, 0.24, 0.68, 0.18, 0.32, 0.28, 0.29, 0.51, 0.27, 0.43, 0.19]
0.029896053430267435
[[ 19.     27.9    43.     39.     39.     43.     -0.85   -0.38   -1.23
  104.9   105.7    -0.8    97.4     0.295   0.309   0.541   0.504  14.2
   23.6     0.215   0.507  13.8    76.1     0.179]
 [ 12.     27.4    42.     30.     40.     32.      1.79    0.01    1.81
  117.8   116.      1.8    98.4     0.238   0.448   0.577   0.54    9.9
   23.      0.195   0.546  11.2    77.5     0.203]
 [  6.     26.1    47.     25.     47.     25.      4.93   -0.11    4.82
  117.1   112.1     5.     97.1     0.219   0.383   0.588   0.557  12.1
   24.7     0.176   0.545  12.9    78.9     0.2  ]
 [  6.     29.7    53.     29.     53.     29.      4.28   -0.15    4.13
  108.3   103.8     4.5    95.8     0.318   0.324   0.556   0.524  12.1
   20.1     0.22    0.48  

In [9]:
dataset.shape

(112, 26)

In [10]:

from IPython.display import display
with pd.option_context('display.max_columns', 250):
    display(non_str_data)

,Rk,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,DRtg,NRtg,Pace,FTr,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,deFG%,dTOV%,dDRB%,dFT/FGA,Attend.,Attend./G,Playoff Wins,Playoff Win Percentage
Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,28.5,52.0,20.0,55,17,9.25,-0.29,8.97,117.6,108.3,9.3,98.5,0.244,0.488,0.597,0.563,12.7,24.5,0.195,0.507,10.3,79.3,0.159,151300.0,4203.0,6,0.545
1,2.0,28.8,47.0,25.0,49,23,6.18,-0.16,6.02,117.6,111.2,6.4,96.9,0.222,0.400,0.599,0.564,12.2,22.7,0.186,0.531,11.9,79.1,0.186,13901.0,386.0,10,0.526
2,3.0,26.6,51.0,21.0,49,23,5.82,-0.15,5.67,117.2,111.3,5.9,97.2,0.212,0.392,0.597,0.564,11.5,20.8,0.177,0.534,12.4,78.5,0.194,104027.0,2890.0,14,0.636
3,4.0,28.1,46.0,26.0,48,24,5.89,-0.32,5.57,117.2,111.4,5.8,102.2,0.233,0.404,0.593,0.566,12.0,23.3,0.177,0.536,11.5,79.7,0.157,64780.0,1799.0,16,0.696
4,5.0,27.1,49.0,23.0,48,24,5.58,-0.31,5.28,113.2,107.6,5.6,99.5,0.293,0.347,0.579,0.541,12.8,23.2,0.225,0.521,13.8,78.2,0.200,68583.0,1905.0,7,0.583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,13.0,25.3,45.0,37.0,43,39,0.79,0.34,1.13,108.2,107.3,0.9,91.4,0.263,0.233,0.537,0.501,12.5,27.1,0.198,0.496,11.7,75.1,0.195,683757.0,16677.0,0,0.000
108,14.0,28.6,46.0,36.0,43,39,0.72,-0.55,0.17,103.7,103.0,0.7,93.7,0.259,0.203,0.534,0.499,14.0,24.9,0.192,0.481,12.9,77.3,0.210,747796.0,18239.0,6,0.600
109,15.0,24.6,41.0,41.0,42,40,0.43,-0.52,-0.09,102.7,102.2,0.5,94.1,0.258,0.223,0.536,0.499,15.5,25.4,0.195,0.487,15.9,73.3,0.221,593910.0,14813.0,2,0.333


In [11]:
dataset = loadtxt('/content/drive/MyDrive/Colab Notebooks/non_str_data_2.csv',delimiter=',',skiprows=1,usecols=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,27,28])
X2 = dataset[1:,:24]
Y2 = dataset[1:,24:25]


In [12]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, Y2, test_size=0.25, random_state=42) #Model 2 for playoff wins

In [13]:
model2 = XGBRegressor()
model2.fit(X2_train, y2_train)
print(model2)

[06:01:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor()


In [14]:
from sklearn.metrics import mean_squared_error
y2_pred = model2.predict(X2_test)
predictions2 = [round(value,2) for value in y2_pred]
print(predictions2)
print(mean_squared_error(y2_test,predictions2,squared=False))
print(y2_test)
print(X2_test)

[1.2, 2.46, 12.39, 6.04, 7.88, 4.17, 2.42, 1.2, 4.49, 8.87, 6.85, 14.05, 12.18, 10.22, 10.09, 8.3, 3.54, 4.59, 0.56, 3.81, 1.41, 2.41, 11.48, 6.45, 5.19, 5.31, 7.58, 1.86]
3.313891756128531
[[ 2.]
 [ 2.]
 [ 4.]
 [ 2.]
 [ 8.]
 [ 4.]
 [ 1.]
 [ 1.]
 [ 7.]
 [ 1.]
 [ 6.]
 [15.]
 [10.]
 [11.]
 [ 7.]
 [ 9.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 3.]
 [ 1.]
 [ 5.]
 [ 6.]
 [ 0.]
 [ 5.]
 [10.]
 [11.]
 [ 2.]]
[[ 19.     27.9    43.     39.     39.     43.     -0.85   -0.38   -1.23
  104.9   105.7    -0.8    97.4     0.295   0.309   0.541   0.504  14.2
   23.6     0.215   0.507  13.8    76.1     0.179]
 [ 12.     27.4    42.     30.     40.     32.      1.79    0.01    1.81
  117.8   116.      1.8    98.4     0.238   0.448   0.577   0.54    9.9
   23.      0.195   0.546  11.2    77.5     0.203]
 [  6.     26.1    47.     25.     47.     25.      4.93   -0.11    4.82
  117.1   112.1     5.     97.1     0.219   0.383   0.588   0.557  12.1
   24.7     0.176   0.545  12.9    78.9     0.2  ]
 [  6.     29.7    53.   